In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
import joblib
from sklearn.linear_model import ElasticNet

In [83]:
df = pd.read_csv("../data/processed/final_data.csv")

In [84]:
ordinal_features = [' khoa', ' hedt', ' chuyennganh2']

df[ordinal_features] = df[ordinal_features].fillna('Unknown')

encoder = OrdinalEncoder()
df_encoded = encoder.fit_transform(df[ordinal_features])

df_encoded = pd.DataFrame(df_encoded, columns=[f"{col}_mahoa" for col in ordinal_features], index=df.index)

df = pd.concat([df, df_encoded], axis=1)

df.drop(columns=ordinal_features, inplace=True)


print(df.head())

                                       mssv  hocky  namhoc  dtbhk  sotchk  \
0  0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6    2.0  2021.0   8.28    21.0   
1  0000AC05XPvAibaEXe9B2tolTZ0JLoBGbkQixQS6    1.0  2022.0   7.56    16.0   
2  0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G    2.0  2019.0   9.00    21.0   
3  0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G    1.0  2020.0   9.11    15.0   
4  0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G    2.0  2020.0   8.75    19.0   

        id   namsinh   gioitinh     noisinh      lopsh   khoahoc   tinhtrang  \
0      NaN       NaN        NaN         NaN        NaN       NaN         NaN   
1      NaN       NaN        NaN         NaN        NaN       NaN         NaN   
2  18570.0    2001.0        0.0   'Nghệ An'   TMĐT2019      14.0         1.0   
3  18570.0    2001.0        0.0   'Nghệ An'   TMĐT2019      14.0         1.0   
4  18570.0    2001.0        0.0   'Nghệ An'   TMĐT2019      14.0         1.0   

     diachi_tinhtp Column1  dtbhk2   khoa_mahoa   hedt_m

In [85]:
# Xử lý NaN
df = df.fillna(df.median(numeric_only=True))
df = df.fillna(df.mode().iloc[0])

# Chọn biến đầu vào và đầu ra
X = df.drop(columns=['nhom', 'dtbhk', 'id', ' namsinh', 'mssv', ' noisinh', ' diachi_tinhtp', 'Column1', ' lopsh'], errors='ignore')  # Xóa cột không cần thiết
y = df['dtbhk']

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
# Khởi tạo mô hình ElasticNet (L1_ratio là tỉ lệ giữa Lasso và Ridge)
elastic_net_model = ElasticNet(alpha=0.2, l1_ratio=0.3, random_state=42)

# Huấn luyện mô hình
elastic_net_model.fit(X_train, y_train)

ElasticNet(alpha=0.2, l1_ratio=0.3, random_state=42)

In [88]:
# tập Test
y_pred = elastic_net_model.predict(X_test)

# Đánh giá
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")

Mean Absolute Error (MAE): 0.928411627465532
Mean Squared Error (MSE): 1.8248530945228663
Root Mean Squared Error (RMSE): 1.350871235359931
R^2 Score: 0.5088550106519965


In [89]:
# Lấy hệ số hồi quy (tầm quan trọng của các đặc trưng)
feature_importances = elastic_net_model.coef_

# Tạo DataFrame để xem tầm quan trọng của từng đặc trưng
import pandas as pd

feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', key=abs, ascending=False)

print(feature_importance_df)

               Feature  Importance
6               dtbhk2    0.705443
5            tinhtrang   -0.115990
9   chuyennganh2_mahoa   -0.001734
0                hocky   -0.000000
1               namhoc    0.000000
2               sotchk    0.000000
3             gioitinh   -0.000000
4              khoahoc   -0.000000
7           khoa_mahoa    0.000000
8           hedt_mahoa    0.000000


In [90]:
test_row = pd.DataFrame({
    'hocky': [2.0],
    'namhoc': [2021.0],
    'sotchk': [21.0],
    ' gioitinh': [1.0],
    ' khoahoc': [11.0],
    ' tinhtrang': [1.0],
    'dtbhk2': [5],
    ' khoa_mahoa': [2.0],
    ' hedt_mahoa': [1.0],
    ' chuyennganh2_mahoa': [4.0]
})

In [91]:
predicted_dtb = elastic_net_model.predict(test_row)
print(f"Dự đoán DTBHK: {predicted_dtb[0]:.2f}")

Dự đoán DTBHK: 5.62


In [92]:
joblib.dump(elastic_net_model, '../models/elastic_net_model.pkl')

['../models/elastic_net_model.pkl']